In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import os

import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

from sklearn.metrics import confusion_matrix

import seaborn as sn
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, save_model, load_model

In [ ]:
input_folder = "path_to_folder"

In [ ]:
good_set = "path_to_good_set"

In [ ]:
bad_set = "path_to_bad_set"

1. Conversion funtion

In [ ]:
def collection(input_folder):
    
    collection_input_data = []
    
    for file in os.listdir(input_folder):
        if file.endswith(".png"):
            print(file)
            temp_img = Image.open(input_folder+file).convert('LA')
            grey_array = np.array(temp_img.getdata())
            grey_array_sorted = np.sort(grey_array, axis = 0)
            grey_array_compact = np.array(np.unique(grey_array_sorted[:,0],return_counts=True)).T
            
            grey_array_add = grey_array_compact
            
            for i in range(0, 256, 1):
                if (i in grey_array_add[:,0]) == False:
                    grey_array_add = np.insert(grey_array_add, i, np.array([i,0]), 0)
                    
            new2 = []
            total = grey_array_compact[:,1].sum()
            for i in range(0,len(grey_array_add),1):
                    #a = grey_array_add[i:i+10,1].sum()
                    b = i
                    c = grey_array_add[i:i+10,1].sum()/total
                    temp = new2.append((c))
                    #new2 = np.vstack((new2, temp))
                    #print(total)     
            
            collection_input_data.append(new2)
    collection_input_data = np.array(collection_input_data) 
    print("made")

    return collection_input_data

2. Data label funtion

In [ ]:
def label_data(input_data, type_data, length):
    labels = np.array([type_data] * length)
    input_data2 = [input_data, labels]
    return input_data2

3. Make good data and label it

In [ ]:
good_data = collection(input_folder+good_set)

In [ ]:
part_one = label_data(good_data, 0, good_data.shape[0])

4. Make bad data and label it

In [ ]:
bad_data = collection(input_folder+bad_set)

In [ ]:
part_two = label_data(bad_data, 1, bad_data.shape[0])

4. Create overall data set and split

In [ ]:
all_data = [np.concatenate((part_one[0], part_two[0]), axis = 0), np.concatenate((part_one[1], part_two[1]), axis = 0)]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(all_data[0], all_data[1], test_size = 0.3)

5. Build model and run it

In [ ]:
model = Sequential()

In [ ]:
#model.add(Flatten(input_shape = x_train[0].shape))

In [ ]:
model.add(Dense(20, activation ='relu', input_shape = x_train[0].shape)) 

In [ ]:
model.add(Dense(1, activation ='softmax'))

In [ ]:
model.summary() 

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy",
             optimizer = "adam",
             metrics = ["accuracy"])

In [ ]:
model.fit(x_train, y_train, epochs=100)

In [ ]:
model.evaluate(x_test, y_test)

6. View results

In [ ]:
yp = model.predict(x_test)

In [ ]:
cm = confusion_matrix(y_test, yp)

In [ ]:
plt.figure(figsize = (10,7))
sn.heatmap(cm, annot = True, fmt="g")
plt.xlabel("Pred")
plt.ylabel("True")

7. Save model

In [ ]:
filepath = "path"

In [ ]:
model.save("save_hdf5")